# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [4]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [5]:
sinasc.head()

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025


In [7]:
print(f"Número de registros: {len(sinasc)}")
print(f"Número de registros únicos: {len(sinasc.drop_duplicates())}")

Número de registros: 27028
Número de registros únicos: 27028


In [8]:
# 2) seu código aqui
valores_missing = sinasc.isnull().sum()
print(valores_missing[valores_missing > 0])

CODESTAB        115
ESTCIVMAE       317
ESCMAE          312
CODOCUPMAE     2907
QTDFILVIVO     1573
QTDFILMORT     2098
GESTACAO       1232
GRAVIDEZ         79
PARTO            49
HORANASC         21
SEXO              4
APGAR1           96
APGAR5          103
RACACOR         647
IDANOMAL        591
CODANOMAL     26814
DTRECORIGA    27028
NATURALMAE      298
CODMUNNATU      298
CODUFNATU       298
ESCMAE2010      249
SERIESCMAE    12710
DTNASCMAE        40
RACACORMAE      661
QTDGESTANT     1212
QTDPARTNOR     1879
QTDPARTCES     1791
IDADEPAI      19421
DTULTMENST    10072
SEMAGESTAC     1238
TPMETESTIM     1238
CONSPRENAT      930
MESPRENAT      2867
TPAPRESENT      265
STTRABPART      947
STCESPARTO      747
TPNASCASSI       61
TPFUNCRESP       67
TPDOCRESP        14
DTDECLARAC       52
ESCMAEAGR1      249
munResLat         1
munResLon         1
munResAlt         1
munResArea        1
dtype: int64


In [21]:
# 3) seu código aqui
colunas = [
    'LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'
]

sinasc_2 = sinasc[colunas]
print(sinasc_2.isnull().sum())

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64


In [22]:
# 4) seu código aqui
sinasc_2 = sinasc_2[sinasc_2['APGAR5'].notna()]

In [23]:
print(f"Número de linhas após remoção: {len(sinasc_2)}")
print(sinasc_2.isnull().sum())

Número de linhas após remoção: 26925
LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64


In [24]:
sinasc_2.head()

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0


In [28]:
# 5) seu código aqui
sinasc_2['ESTCIVMAE'] = sinasc_2['ESTCIVMAE'].fillna(9)
sinasc_2['CONSULTAS'] = sinasc_2['CONSULTAS'].fillna(9)

In [29]:
sinasc.head()

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,<NA>,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025


In [25]:
# 6) Seu código aqui
sinasc_2['QTDFILVIVO'] = sinasc_2['QTDFILVIVO'].fillna(0)

In [26]:
# 7) seu código aqui
sinasc_2['GESTACAO'] = sinasc_2['GESTACAO'].fillna(9)
sinasc_2['GRAVIDEZ'] = sinasc_2['GRAVIDEZ'].fillna(9)
sinasc_2['ESTCIVMAE'] = sinasc_2['ESTCIVMAE'].fillna(9)
sinasc_2['ESCMAE'] = sinasc_2['ESCMAE'].fillna(9)

In [30]:
sinasc_2.head()

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,APGAR5_CAT
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal


In [40]:
# 8) seu código aqui
sinasc_2['APGAR5_CAT'] = pd.NA
sinasc_2.loc[sinasc_2['APGAR5'] <= 3, 'APGAR5_CAT'] = 'asfixia_grave'
sinasc_2.loc[(sinasc_2['APGAR5'] >= 4) & (sinasc_2['APGAR5'] <= 5), 'APGAR5_CAT'] = 'asfixia_moderada'
sinasc_2.loc[(sinasc_2['APGAR5'] >= 6) & (sinasc_2['APGAR5'] <= 7), 'APGAR5_CAT'] = 'asfixia_leve'
sinasc_2.loc[sinasc_2['APGAR5'] >= 8, 'APGAR5_CAT'] = 'normal'

sinasc_2['APGAR5_CAT'].head(21)

0            normal
1            normal
2            normal
3            normal
4            normal
5            normal
6            normal
7            normal
8            normal
9            normal
10           normal
11           normal
12           normal
13           normal
14     asfixia_leve
15           normal
16           normal
17           normal
18           normal
19           normal
20    asfixia_grave
Name: APGAR5_CAT, dtype: object

In [41]:
# 9) seu código aqui
sinasc_2.rename(columns={
  'LOCNASC': 'loc_nasc',
  'IDADEMAE': 'idade_mae',
  'ESTCIVMAE': 'est_civ_mae',
  'ESCMAE': 'esc_mae',
  'QTDFILVIVO': 'qtd_fil_vivo',
  'GESTACAO': 'gestacao',
  'GRAVIDEZ': 'gravidez',
  'CONSULTAS': 'consultas',
  'APGAR5': 'apgar5'
}, inplace=True)

In [42]:
sinasc_2.head()

,loc_nasc,idade_mae,est_civ_mae,esc_mae,qtd_fil_vivo,gestacao,gravidez,consultas,apgar5,APGAR5_CAT
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
